<a href="https://colab.research.google.com/github/fBurde/NtlM-React-Express-Mysql/blob/master/ChatbotFBurde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
data ={"intents": [
                   

      {"tag": "greeting",
       "patterns": ["hello","hey","hi","good day","how is it going?","welcome","anybody there?"],
       "responses": ["Hello!","Hey!","how can i help you?","whats the reason for your visit today","greetings, what is your problem?"]},

       {"tag": "goodbye",
       "patterns": ["goodbye","bye","farewell","see you later","smell you later","thanks for the help","you have helpted me"],
       "responses": ["goodbye!","farewell!","see you","if you have no further Problems, i bid you farewell"]},
      
       {"tag": "age",
       "patterns": ["how old are you?"],
       "responses": ["I am an bot therefore i have no age, i am online since 02.09.2021"]},

      {"tag": "name",
       "patterns": ["What is your name?","what are you called?"],
       "responses": ["My name is Felix, i am a artifical intelligence bot. I can help you with alot of questions."]},


      {"tag": "stocks",
       "patterns": ["How are my stocks doing?","What stocks are rising?"],
       "responses": ["Stocks are doing fine, or are they?","to The Moon!!!....."]},


      {"tag": "cancelation",
       "patterns": ["i dont want this anymore","I want to cancel my service","this service is stupid","I hate this service"],
       "responses": ["I can help you with that. What is the reason for your cancelation?","May i ask why you want to cancel your services?"]},


      {"tag": "service",
       "patterns": ["I have some questions regarding services","im unhappy with my service"],
       "responses": ["I can help you with that. What is your question?","How can i help you?"]},


      {"tag": "purchase",
       "patterns": ["I want to buy a new package","I want to upgrade my service","i want to purchase a new item","acquire this now"],
       "responses": ["in what kind of package are you interested?","please select a package you are interested in: 1.Upgrade 2.New Customer 3.new Service"]},


      {"tag": "Password",
       "patterns": ["I have forgotten my password","i can't login","it's not working","I am not getting in"],
       "responses": ["I can help you with that","if you would like to open a ticket please enter your email connected to your account", "do you want to send an E-Mail to let the it department manually reset your passwor"]},
]}

In [2]:
#print(data)

import random
import json
import pickle
import numpy as np

import nltk 
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

intents = data

#print(intents)

words = []
classes = []
documents = []
ignore_letters = ['?','!','.'',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append(((word_list),intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


# WORDS NOW AS NUMERICAL ValueError
# BAGOF WORDS USED 

print(words)
print(classes)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
["'s", 'How', 'I', 'What', 'a', 'acquire', 'am', 'anybody', 'anymore', 'are', 'buy', 'bye', 'ca', 'called', 'cancel', 'day', 'doing', 'dont', 'farewell', 'for', 'forgotten', 'getting', 'going', 'good', 'goodbye', 'hate', 'have', 'hello', 'help', 'helpted', 'hey', 'hi', 'how', 'i', 'im', 'in', 'is', 'it', 'item', 'later', 'login', 'me', 'my', "n't", 'name', 'new', 'not', 'now', 'old', 'package', 'password', 'purchase', 'question', 'regarding', 'rising', 'see', 'service', 'smell', 'some', 'stock', 'stupid', 'thanks', 'the', 'there', 'this', 'to', 'unhappy', 'upgrade', 'want', 'welcome', 'what', 'with', 'working', 'you', 'your']
['Password', 'age', 'cancelation', 'goodbye', 'greeting', 'name', 'purchase', 'service', 'stocks']


TRAINING ENGAGE!!!!!!!!!!!

In [27]:
training = []
output_empty = [0] *len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)


  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])


random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5,verbose=1)
model.save('chatbotmodel.h5',hist)
print("Done!!!!!! YEA")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
7/7 [==============================] - 0s 4ms/step - loss: 2.3167 - accuracy: 0.0303
Epoch 2/200
7/7 [==============================] - 0s 4ms/step - loss: 2.1827 - accuracy: 0.1212
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 2.1525 - accuracy: 0.1212
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 2.0658 - accuracy: 0.4242
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 1.9468 - accuracy: 0.4545
Epoch 6/200
7/7 [==============================] - 0s 5ms/step - loss: 1.8469 - accuracy: 0.4545
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 1.7188 - accuracy: 0.5455
Epoch 8/200
7/7 [==============================] - 0s 6ms/step - loss: 1.7023 - accuracy: 0.4848
Epoch 9/200
7/7 [==============================] - 0s 4ms/step - loss: 1.6218 - accuracy: 0.5152
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 1.4522 - accuracy: 0.5152
Epoch 11/200
7/7 [===========

**CHATBOT TIME**

In [20]:
#
#
#
#

import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbotmodel.h5')


def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words 


def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_TRESHOLD = 0.25
  results = [[i,r] for i, r in enumerate(res) if r > ERROR_TRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  print(results)
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    #print(idx)
    #print(i)
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result





In [31]:
#print(intents)
asking = True

while asking:
  message =input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  if(ints[0]["intent"] == "goodbye"):
    asking = False
  #print(message)
  print(res)

stocks
[[8, 0.6965668]]
Stocks are doing fine, or are they?
purchase
[[3, 0.73995656]]
if you have no further Problems, i bid you farewell


In [32]:
asking = True

while asking:
  message =input("")
  if "bye" in message:
    print("this is goodbye, dont be sad")
    asking = False
    break
  if "purchase" in message:
    print("please choose one option: 1.servicevertrag verlängern 2.Neukunde 3.andere Abzockangebote")
    message =input("")
    if "1" or "servicevertrag verlängern" in message:
      print("please enter your Accountnumber:")
      accountnumber =input("")
      print(f"is this the correct number? Number: {accountnumber}")
  if  "hello" in message:
    print("Hello")
  else: 
    print("no valid input %s".format())

KeyboardInterrupt: ignored